In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.preprocessing import StandardScaler, MinMaxScaler



from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,f1_score

from tqdm import tqdm
from sklearn.utils.fixes import delayed
from joblib import Parallel, parallel_backend


In [2]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#若想本地运行，此处需要修改
df_train=pd.read_csv('training_set_VU_DM.csv')
df_test=pd.read_csv('test_set_VU_DM.csv')

clean df

In [3]:
def Median_padding(df:DataFrame,column:str):
    # 计算众数并确保众数计算成功
    if not df[column].mode().empty:
        mode_value = df[column].mode()[0]
        print(column+"众数值:", mode_value)

        # 用众数填充空缺值
        df[column].fillna(mode_value, inplace=True)

        # 检查填充后的空缺值情况
        print(column+"填充后的空缺值个数:", df[column].isnull().sum())
    else:
        print(column+"无法计算众数，因为数据列中没有有效值。")
        

def fill_missing_values(df:DataFrame):#严格按照每个酒店的中位数进行填充，但是太慢了，放弃
    prop_id=df['prop_id'].unique()
    fill_cols=['comp1_rate','comp1_inv','comp1_rate_percent_diff',
                'comp2_rate','comp2_inv','comp2_rate_percent_diff',
                'comp3_rate','comp3_inv','comp3_rate_percent_diff',                
                'comp4_rate','comp4_inv','comp4_rate_percent_diff',
                'comp5_rate','comp5_inv','comp5_rate_percent_diff',
                'comp6_rate','comp6_inv','comp6_rate_percent_diff',
                'comp7_rate','comp7_inv','comp7_rate_percent_diff',
                'comp8_rate','comp8_inv','comp8_rate_percent_diff']
    #用一个字典存储每个属性的中位数
    comp_median = {col: df[col].median() for col in fill_cols}
    
    temp = df[['prop_id'] + fill_cols].copy()

    

    def process_group(prop):
        prop_df = temp[temp['prop_id'] == prop]
        for col in fill_cols:
            if prop_df[col].isna().all():  # 如果这一列全是空值,则用comp_median[col]填充
                fill_value = comp_median[col]
            else:  # 如果这一列不全是空值,则用这一列的中位数填充
                fill_value = prop_df[col].median()
            temp.loc[temp['prop_id'] == prop, col] = prop_df[col].fillna(fill_value)
    
    # 使用 Parallel 和 delayed 并行处理
    Parallel(n_jobs=-1,backend='threading',verbose=1)(delayed(process_group)(prop) for prop in prop_id)
    
    df.update(temp)


        



In [4]:
fill_cols_train=['comp1_rate','comp1_inv','comp1_rate_percent_diff',
                'comp2_rate','comp2_inv','comp2_rate_percent_diff',
                'comp3_rate','comp3_inv','comp3_rate_percent_diff',                
                'comp4_rate','comp4_inv','comp4_rate_percent_diff',
                'comp5_rate','comp5_inv','comp5_rate_percent_diff',
                'comp6_rate','comp6_inv','comp6_rate_percent_diff',
                'comp7_rate','comp7_inv','comp7_rate_percent_diff',
                'comp8_rate','comp8_inv','comp8_rate_percent_diff',
                'visitor_hist_starrating','visitor_hist_adr_usd','prop_review_score',
                'prop_location_score2','srch_query_affinity_score','orig_destination_distance','gross_bookings_usd']

fill_cols_test=['comp1_rate','comp1_inv','comp1_rate_percent_diff',
                'comp2_rate','comp2_inv','comp2_rate_percent_diff',
                'comp3_rate','comp3_inv','comp3_rate_percent_diff',                
                'comp4_rate','comp4_inv','comp4_rate_percent_diff',
                'comp5_rate','comp5_inv','comp5_rate_percent_diff',
                'comp6_rate','comp6_inv','comp6_rate_percent_diff',
                'comp7_rate','comp7_inv','comp7_rate_percent_diff',
                'comp8_rate','comp8_inv','comp8_rate_percent_diff',
                'visitor_hist_starrating','visitor_hist_adr_usd','prop_review_score',
                'prop_location_score2','srch_query_affinity_score','orig_destination_distance']

for col in fill_cols_train:
    Median_padding(df_train,col)
for col in fill_cols_test:
    Median_padding(df_test,col)

comp1_rate众数值: 1.0
comp1_rate填充后的空缺值个数: 0
comp1_inv众数值: 0.0
comp1_inv填充后的空缺值个数: 0
comp1_rate_percent_diff众数值: 9.0
comp1_rate_percent_diff填充后的空缺值个数: 0
comp2_rate众数值: 0.0
comp2_rate填充后的空缺值个数: 0


C:\Users\sydwi\AppData\Local\Temp\ipykernel_13316\1420989824.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mode_value, inplace=True)


comp2_inv众数值: 0.0
comp2_inv填充后的空缺值个数: 0
comp2_rate_percent_diff众数值: 11.0
comp2_rate_percent_diff填充后的空缺值个数: 0
comp3_rate众数值: 0.0
comp3_rate填充后的空缺值个数: 0
comp3_inv众数值: 0.0
comp3_inv填充后的空缺值个数: 0
comp3_rate_percent_diff众数值: 11.0
comp3_rate_percent_diff填充后的空缺值个数: 0
comp4_rate众数值: 0.0
comp4_rate填充后的空缺值个数: 0
comp4_inv众数值: 0.0
comp4_inv填充后的空缺值个数: 0
comp4_rate_percent_diff众数值: 9.0
comp4_rate_percent_diff填充后的空缺值个数: 0
comp5_rate众数值: 0.0
comp5_rate填充后的空缺值个数: 0
comp5_inv众数值: 0.0
comp5_inv填充后的空缺值个数: 0
comp5_rate_percent_diff众数值: 11.0
comp5_rate_percent_diff填充后的空缺值个数: 0
comp6_rate众数值: 0.0
comp6_rate填充后的空缺值个数: 0
comp6_inv众数值: 0.0
comp6_inv填充后的空缺值个数: 0
comp6_rate_percent_diff众数值: 11.0
comp6_rate_percent_diff填充后的空缺值个数: 0
comp7_rate众数值: 0.0
comp7_rate填充后的空缺值个数: 0
comp7_inv众数值: 0.0
comp7_inv填充后的空缺值个数: 0
comp7_rate_percent_diff众数值: 11.0
comp7_rate_percent_diff填充后的空缺值个数: 0
comp8_rate众数值: 0.0
comp8_rate填充后的空缺值个数: 0
comp8_inv众数值: 0.0
comp8_inv填充后的空缺值个数: 0
comp8_rate_percent_diff众数值: 10.0
comp8_rate_percent_dif

feature engineering

In [11]:
def feature_eng(df):


    # 删除原始时间戳列
    if 'date_time' in df:
        df['date_time'] = pd.to_datetime(df['date_time'])
        # 提取基本成分
        df['year'] = df['date_time'].dt.year
        df['month'] = df['date_time'].dt.month
        df['day'] = df['date_time'].dt.day
        df['hour'] = df['date_time'].dt.hour

        df = df.drop('date_time', axis=1)
    
    if 'booking_bool' in df:
        df['booking_bool'] = df['booking_bool'].astype('float32')
    
    if 'click_bool' in df:
        df= df.drop('click_bool', axis=1)
    
    if 'position' in df:
        df = df.drop('position', axis=1)
    
    if 'gross_bookings_usd' in df:
        df = df.drop('gross_bookings_usd', axis=1)
        

    # 标准化和归一化float变量
    # 选择要标准化的列，排除 'prop_id' 和 'srch_id'
    columns_to_scale = df.columns.difference(['prop_id', 'srch_id'])

    # 初始化 StandardScaler
    scaler = StandardScaler()

    # 对选定的列进行标准化
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
    
            
    # 将所有 float64 类型转换为 float32，所有 int64 类型转换为 int32
    df = df.astype({col: 'float32' for col in df.select_dtypes(include='float64').columns}) 
    df = df.astype({col: 'int32' for col in df.select_dtypes(include='int64').columns})
    
    
    return df

df_train = feature_eng(df_train)
df_test = feature_eng(df_test)

In [15]:
# 将 df_train 保存到 CSV 文件
df_train.to_csv('df_train_std.csv', index=False)
df_test.to_csv('df_test_std.csv', index=False)


In [14]:
any_nan = df_train.isnull().values.any()
print("Does the DataFrame have any NaN values?")
print(any_nan)
any_nan = df_test.isnull().values.any()
print("Does the DataFrame have any NaN values?")
print(any_nan)

Does the DataFrame have any NaN values?
False
Does the DataFrame have any NaN values?
False


Train

In [12]:
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 54 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int32  
 1   site_id                      float32
 2   visitor_location_country_id  float32
 3   visitor_hist_starrating      float32
 4   visitor_hist_adr_usd         float32
 5   prop_country_id              float32
 6   prop_id                      int32  
 7   prop_starrating              float32
 8   prop_review_score            float32
 9   prop_brand_bool              float32
 10  prop_location_score1         float32
 11  prop_location_score2         float32
 12  prop_log_historical_price    float32
 13  price_usd                    float32
 14  promotion_flag               float32
 15  srch_destination_id          float32
 16  srch_length_of_stay          float32
 17  srch_booking_window          float32
 18  srch_adults_count            float32
 19  

In [18]:

df_train_sampled = df_train.sample(frac=0.01, random_state=42)
features=df_train_sampled.drop(['prop_id','srch_id'],axis=1)
target = df_train_sampled['booking_bool']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features,target,test_size=0.3)

# 实例化随机森林回归器
model = RandomForestRegressor(random_state=1)
param_grid = {
    'n_estimators': [50, 100, 200],  # 树的数量
    'max_features': ['sqrt', 'log2'],  # 考虑的最大特征数
    'max_depth': [10, 20, 30],  # 最大树深度
    'min_samples_split': [2, 5, 10],  # 分割内部节点所需的最小样本数
    'min_samples_leaf': [1, 2, 4]  # 叶节点所需的最小样本数
}

# 创建 RandomizedSearchCV 对象
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=50, cv=3, n_jobs=8, scoring='r2', verbose=1, random_state=42)

# 拟合随机搜索
random_search.fit(Xtrain, Ytrain)
print("Best parameters found: ", random_search.best_params_)
best_rf = random_search.best_estimator_
val_predictions = best_rf.predict(Xtest)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best parameters found:  {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30}


In [19]:
mse = mean_squared_error(Ytest, val_predictions)
print(f'Mean Squared Error (MSE): {mse}')

# 计算RMSE
rmse = mean_squared_error(Ytest, val_predictions, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse}')

# 计算MAE
mae = mean_absolute_error(Ytest, val_predictions)
print(f'Mean Absolute Error (MAE): {mae}')

# 计算R^2
r2 = r2_score(Ytest, val_predictions)
print(f'R^2: {r2}')

Mean Squared Error (MSE): 0.00926889780822005
Root Mean Squared Error (RMSE): 0.09627511520751378
Mean Absolute Error (MAE): 0.02931294083633428
R^2: 0.9905504059214852


c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
